In [1]:

from itertools import product
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
import os
os.chdir('..')
from src.swbm import *
from src.plots import *
from src.utils import *



In [16]:
# Values for each variable

cs = [210, 420, 840]
b0 = [0.4, 0.6, 0.8]
g = [0.2, 0.5, 0.8]
a = [2, 4, 8]

# Generate all combinations using itertools.product
all_combinations = list(product(cs, b0, g, a))

In [17]:
# Load and pre-process data
input_swbm_raw = pd.read_csv('data/Data_swbm_Germany.csv')
input_swbm = prepro(input_swbm_raw)

In [92]:
# test
test_combinations = all_combinations[:5]
swbm_param = 'b0'
np.random.seed(42)
results_all = []


for combination in test_combinations:
    # Run SWBM without seasonal variation
    #break
    const_swbm_params = {'c_s': combination[0], 
                         'b0': combination[1], 
                         'g': combination[2], 
                         'a': combination[3]}

    # optimize sinus params for b0
    init_values = [0.5, 2, 5, combination[1]]

    res = minimize(opt_swbm_corr,
                np.asarray(init_values).flatten(),  # has to be 1D
                args=(input_swbm, const_swbm_params, swbm_param),
                options={"maxiter": 500, "disp": False})
    opt_params_df = minimize_res2df(res, [swbm_param])
    
    # Set swbm const_swbm_params
    params_seasonal = {'c_s': combination[0], 
                         'b0': combination[1], 
                         'g': combination[2], 
                         'a': combination[3]}

    # Get sinus curve for current single parameter
    params_seasonal[swbm_param] = seasonal_sinus(
        len(input_swbm),
        amplitude=opt_params_df.loc['amplitude', swbm_param],
        freq=opt_params_df.loc['freq', swbm_param],
        phase=opt_params_df.loc['phase', swbm_param],
        center=opt_params_df.loc['center', swbm_param],
        which=swbm_param
    )

    # Run SWBM with optimized seasonal b0
    preds_seasonal = predict_ts(input_swbm, params_seasonal)
    moists_seasonal, runoffs_seasonal, ets_seasonal = preds_seasonal

    # calculate correlation
    eval_df = eval_swbm(input_swbm,
                        {'sm': moists_seasonal,
                        'ro': runoffs_seasonal,
                        'le': ets_seasonal},
                        swbm_param)

   # save all results in dict 
    combination_dict = {
        'Combination': combination,
        'sum_corr': eval_df['corr'].sum(),
        'SinusParameters': opt_params_df,
        'eval_df': eval_df
    }


    results_all.append(combination_dict)

c:\Users\motze\OneDrive\Master\ESM\VegSWBM\src\swbm.py:5: RuntimeWarning: invalid value encountered in scalar power
  return b0 * (w_i / c_s) ** g


ValueError: array must not contain infs or NaNs

In [79]:
# get dict with highes corr_sum score
max_sum_corr_dict = max(results_all, key=lambda x: x['sum_corr'])
max_sum_corr_dict

In [93]:
combination

(210, 0.4, 0.2, 2)

In [94]:
init_values

[0.5, 2, 5, 0.4]

In [97]:
const_swbm_params = {'c_s': combination[0], 
                         'b0': combination[1], 
                         'g': combination[2], 
                         'a': combination[3]}

const_swbm_params

{'c_s': 210, 'b0': 0.4, 'g': 0.2, 'a': 2}

In [98]:
res = minimize(opt_swbm_corr,
                np.asarray(init_values).flatten(),  # has to be 1D
                args=(input_swbm, const_swbm_params, swbm_param),
                options={"maxiter": 500, "disp": False})
opt_params_df = minimize_res2df(res, [swbm_param])

ValueError: array must not contain infs or NaNs

In [91]:
const_swbm_params = {'c_s': combination[0], 
                         'b0': combination[1], 
                         'g': combination[2], 
                         'a': combination[3]}

{'c_s': 210,
 'b0': array([0.16711537, 0.16636118, 0.16569435, ..., 0.2087652 , 0.20608482,
        0.20348016]),
 'g': 0.5,
 'a': 4}